In [ ]:
# Standard libraries
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Matplotlib customization
import matplotlib as mpl
from matplotlib.colors import ListedColormap, BoundaryNorm

# Scikit-learn tools
from sklearn import metrics
from sklearn.metrics import (
    roc_auc_score, roc_curve, accuracy_score, f1_score, 
    precision_score, recall_score
)
from sklearn.model_selection import (
    GroupKFold, StratifiedKFold, cross_validate, GridSearchCV
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

# PyTorch for deep learning
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# Skorch for PyTorch integration with Scikit-learn
from skorch import NeuralNetClassifier
from skorch.callbacks import LRScheduler

# UMAP for dimensionality reduction
from umap import UMAP

# SHAP for model interpretability
import shap

# Plotly for interactive visualization
import plotly.express as px

# Custom modules
from tmaR import classifiers, FeatureProcessing

In [ ]:
class StratifiedGroupKFold:
    def __init__(self, n_splits=5, shuffle=False, random_state=None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y, groups):
        """
        Stratified Group K-Fold splitting.
        
        Parameters:
        X: array-like, shape (n_samples, n_features) - Feature matrix.
        y: array-like, shape (n_samples,) - Target labels.
        groups: array-like, shape (n_samples,) - Group identifiers.

        Yields:
        train_idx, test_idx - Indices for train and test splits.
        """
        groups = np.array(groups)  # Ensure groups is a NumPy array
        y = np.array(y)  # Ensure y is a NumPy array
        unique_groups = np.unique(groups)

        if self.shuffle:
            rng = np.random.RandomState(self.random_state)
            rng.shuffle(unique_groups)

        # Assign a label to each group based on the majority label in `y`
        group_labels = {group: np.argmax(np.bincount(y[np.where(groups == group)])) for group in unique_groups}

        stratified_labels = np.array([group_labels[group] for group in groups])

        strat_kfold = StratifiedKFold(n_splits=self.n_splits, shuffle=self.shuffle, random_state=self.random_state)
        
        for train_group_idx, test_group_idx in strat_kfold.split(unique_groups, [group_labels[g] for g in unique_groups]):
            train_groups, test_groups = unique_groups[train_group_idx], unique_groups[test_group_idx]

            train_idx = np.where(np.isin(groups, train_groups))[0]
            test_idx = np.where(np.isin(groups, test_groups))[0]

            yield train_idx, test_idx


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.5):
        """
        Parameters:
            input_dim : int
                Dimensionality of the input (e.g., 143)
            hidden_dim : int
                Size of the hidden layer (must be divisible by num_tokens)
            output_dim : int
                Number of classes (k)
            num_tokens : int (default=8)
                Number of tokens to split the hidden representation into.
            dropout_rate : float (default=0.5)
                Dropout probability to prevent overfitting.
                
        This model is an MLP with:
        - Batch Normalization for stable training
        - Dropout for regularization
        - Fully connected layers for classification
        """
        super(MLP, self).__init__()
        
        
        # First fully connected layer
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout1 = nn.Dropout(dropout_rate)  # Dropout after activation
        
        # Second fully connected layer (classification layer)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # First FC layer with ReLU, Batch Norm, and Dropout
        h = self.fc1(x)  # Linear transformation
        h = F.relu(h)  # Activation function
        h = self.dropout1(h)  # Apply Dropout
        
        # Final classification layer
        out = self.fc2(h)  # (B, output_dim)
        return out


In [ ]:
class FC(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.5):
        """
        A simple MLP with no hidden layer.

        Parameters:
            input_dim : int
                Dimensionality of the input (e.g., 143)
            output_dim : int
                Number of classes (k)
            dropout_rate : float (default=0.5)
                Dropout probability to prevent overfitting.
        """
        super(FC, self).__init__()

        # Dropout before classification
        self.dropout = nn.Dropout(dropout_rate)

        # Directly map input to output
        self.fc_out = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # Apply dropout before the classification layer
        x = self.dropout(x)
        out = self.fc_out(x)  # (B, output_dim)
        return out


#### Diagnosen revidiert kodiert: 
<br>Papillär classic=1, 
<br>Papillär follikuläre Variante=2, 
<br>Papillär poorly diff. Anteil=3, 
<br>Follikulär minimally invasive=4, 
<br>Follikulär widely invasive=5, 
<br>Papilläres Karzinom spezielle Variante=6,
<br>Follikulär onkozytär minimally invasive=7,
<br>Follikulär onkozytär widely invasive=8,
<br>Follikulär poorly diff. Anteil=9,
<br>Follikuläres onkozytäres Adenom=10,
<br>follikulär poorly differentiated onkozytär=11,
<br>Follikuläres Adenom=12
<br>Normal tissue=13
<br>Nodular Hyperplasia=15
<br>Medullary thyroid carcinoma=14

In [ ]:
%store -r processed_radiomics
%store -r meta_data
df = processed_radiomics

# Filter data for tissue type and drop NaN values
label = 'tissue'

df = df.reindex(meta_data.index)

# Filter by tissue type if label is not 'tissue'
if label not in ['tissue']:
    valid_indices = meta_data[meta_data['tissue'] == 1].index
    meta_data = meta_data.loc[valid_indices]
    df = df.loc[valid_indices]

# Drop rows with NaN values in the label column
meta_data = meta_data.dropna(subset=[label])
meta_data = meta_data[~meta_data['ID'].isin([734, 560, 654])]
df = df.loc[meta_data.index]

if label == 'Diagnosis':
    meta_data[label] = meta_data[label].replace({1: 0, 2:0, 6:0}) # remapping some classes to simplify the problem
    meta_data[label] = meta_data[label].replace({4:1,5:1,12:1}) #FTC
    meta_data[label] = meta_data[label].replace({3: 2, 9:2, 11:2})  #Poorly
    meta_data[label] = meta_data[label].replace({7: 3, 8:3, 10:3}) #Onkozytar
    #meta_data[label] = meta_data[label].replace({4:2}) #Onkozytar
    
    # Remove unwanted labels
    exclude_labels = [2,3,13,14,15]
    meta_data = meta_data[~meta_data[label].isin(exclude_labels)]
    meta_data = meta_data.dropna(subset=[label])
    df = df.loc[meta_data.index]
    
if label in ['BRAF', 'BRAF_2nd', 'BRAF_combi']:
    indices = meta_data[meta_data['Diagnosis'].isin([1,2,3,6,7,8,9,10,11,14])].index
    meta_data = meta_data.loc[indices]
    df = df.loc[indices]

if label in ['RAS']:
    #indices = meta_data[meta_data['Diagnosis'].isin([4,5,9,12,2])].index #uncomment for Follicular tumors only
    indices = meta_data[meta_data['Diagnosis'].isin([1,2,3,4,5,6,7,8,9,10,11,12,14])].index #uncomment for including all tumors in RAS classification
    meta_data = meta_data.loc[indices]
    df = df.loc[indices]
    
if label in ['Relapse']:
    meta_data = meta_data[~meta_data['TMA'].isin([2406])]
    df = df.loc[meta_data.index]
    

label_df = meta_data[label].astype(int)

path = f"J:\\TMAs\\Radiomics_v6\\Results_final\\{label}"
os.makedirs(path, exist_ok=True)
os.chdir(path)

In [ ]:
# Get unique classes and their counts
unique_classes, class_counts = np.unique(label_df, return_counts=True)

# Print unique classes
print(f"List of unique classes: {unique_classes}")

# Print number of samples per class
for cls, count in zip(unique_classes, class_counts):
    print(f"Class {cls}: {count} samples")

# Store results in a dictionary (optional)
class_distribution = dict(zip(unique_classes, class_counts))

### Classification

In [ ]:
# ------------------------------
# Training and Model Parameters
# ------------------------------
num_epochs = 10
batch_size = 16
learning_rate = 1e-3
weight_decay = 1e-3
dropout_rate=0.1
gamma = 0.9

n_splits = 5
group_kfold = GroupKFold(n_splits=n_splits)

# Assume df is your features DataFrame and label_df contains your labels.
input_dim = df.shape[1]
hidden_dim = 256 #simpleMLP does not have a hidden layer
output_dim = 2  # binary classification

# Define class weights (if needed) and device
class_weights = torch.tensor([1, 1], dtype=torch.float32)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ------------------------------
# Create the skorch NeuralNetClassifier
# ------------------------------
net = NeuralNetClassifier(
    module=MLP,
    module__input_dim=input_dim,
    module__hidden_dim=hidden_dim,
    module__output_dim=output_dim,
    module__dropout_rate=dropout_rate,
    max_epochs=num_epochs,
    batch_size=batch_size,
    optimizer=optim.Adam,
    optimizer__lr=learning_rate,
    optimizer__weight_decay=weight_decay,
    criterion=nn.CrossEntropyLoss,
    criterion__weight=class_weights,
    device=device,
    # Use an exponential learning rate scheduler callback
    callbacks=[('lr_scheduler', LRScheduler(policy='ExponentialLR', gamma=gamma))],
    # We use external cross-validation so disable internal train/validation split.
    train_split=None,
    verbose=0
)

# ------------------------------
# Create the sklearn Pipeline
# ------------------------------
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('net', net)
])

# ------------------------------
# Data Preparation
# ------------------------------
# Convert data to numpy arrays (assumed float32 for features and int64 for labels)
X = df.values.astype(np.float32)
y = label_df.values.astype(np.int64)
groups = meta_data['PID'].values  # grouping variable

# ------------------------------
# Initialize Lists for Metrics and ROC Curves
# ------------------------------
fold_train_losses     = []
fold_train_aucs       = []
fold_train_accuracies = []
fold_train_f1s        = []
fold_train_precision  = []
fold_train_recall     = []

fold_val_losses       = []
fold_val_aucs         = []
fold_val_accuracies   = []
fold_val_f1s          = []
fold_val_precision    = []
fold_val_recall       = []

roc_curves_train      = []  # list of tuples: (fpr, tpr) per fold
roc_curves_val        = []

all_train_probs_1_list = []
all_val_probs_1_list   = []
all_train_targets_list = []
all_val_targets_list   = []

global_pos_low_confidence = []  # indices of true positives with low confidence (< 0.5)
global_neg_high_confidence = []  # indices of true negatives with high confidence (>= 0.5)

# ------------------------------
# Function to Compute Loss on a Dataset
# ------------------------------
def compute_loss(model, X_data, y_data):
    model.eval()
    X_tensor = torch.tensor(X_data, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y_data, dtype=torch.long).to(device)
    outputs = model(X_tensor)
    loss = nn.CrossEntropyLoss(weight=class_weights)(outputs, y_tensor).item()
    return loss

# ------------------------------
# Cross-Validation Loop
# ------------------------------
for fold, (train_idx, val_idx) in enumerate(group_kfold.split(X, y, groups)):
    print(f"Fold {fold+1}/{n_splits}")
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val     = X[val_idx], y[val_idx]
    
    # Fit the pipeline on the training split.
    pipeline.fit(X_train, y_train)
    
    # The scaler is part of the pipeline so transform X for loss computation.
    X_train_scaled = pipeline.named_steps['scaler'].transform(X_train)
    X_val_scaled   = pipeline.named_steps['scaler'].transform(X_val)
    
    # Get the trained PyTorch model from skorch.
    trained_model = pipeline.named_steps['net'].module_
    
    # Compute training and validation losses.
    train_loss = compute_loss(trained_model, X_train_scaled, y_train)
    val_loss   = compute_loss(trained_model, X_val_scaled, y_val)
    
    # Obtain predictions and probabilities.
    train_probs = pipeline.predict_proba(X_train)
    train_preds = pipeline.predict(X_train)
    val_probs   = pipeline.predict_proba(X_val)
    val_preds   = pipeline.predict(X_val)
    
    # Store targets and probabilities (for later histograms).
    all_train_targets_list.append(y_train)
    all_val_targets_list.append(y_val)
    all_train_probs_1_list.append(train_probs[:, 1])
    all_val_probs_1_list.append(val_probs[:, 1])
    
    # Compute performance metrics.
    train_auc  = roc_auc_score(y_train, train_probs[:, 1])
    train_acc  = accuracy_score(y_train, train_preds)
    train_f1   = f1_score(y_train, train_preds, average='weighted')
    train_prec = precision_score(y_train, train_preds, average='weighted')
    train_rec  = recall_score(y_train, train_preds, average='weighted')
    
    val_auc  = roc_auc_score(y_val, val_probs[:, 1])
    val_acc  = accuracy_score(y_val, val_preds)
    val_f1   = f1_score(y_val, val_preds, average='weighted')
    val_prec = precision_score(y_val, val_preds, average='weighted')
    val_rec  = recall_score(y_val, val_preds, average='weighted')
    
    # Save metrics per fold.
    fold_train_losses.append(train_loss)
    fold_train_aucs.append(train_auc)
    fold_train_accuracies.append(train_acc)
    fold_train_f1s.append(train_f1)
    fold_train_precision.append(train_prec)
    fold_train_recall.append(train_rec)
    
    fold_val_losses.append(val_loss)
    fold_val_aucs.append(val_auc)
    fold_val_accuracies.append(val_acc)
    fold_val_f1s.append(val_f1)
    fold_val_precision.append(val_prec)
    fold_val_recall.append(val_rec)
    
    # Compute ROC curves for both training and validation.
    train_fpr, train_tpr, _ = roc_curve(y_train, train_probs[:, 1])
    roc_curves_train.append((train_fpr, train_tpr))
    val_fpr, val_tpr, _ = roc_curve(y_val, val_probs[:, 1])
    roc_curves_val.append((val_fpr, val_tpr))
    
    # Identify misclassified low-confidence predictions on the validation set.
    pos_low_conf = []  # true positive samples with predicted probability for class 1 < 0.5
    neg_high_conf = [] # true negative samples with predicted probability for class 1 >= 0.5
    for idx, gt, prob in zip(val_idx, y_val, val_probs):
        if gt == 1 and prob[1] < 0.5:
            pos_low_conf.append(idx)
        elif gt == 0 and prob[1] >= 0.5:
            neg_high_conf.append(idx)
    global_pos_low_confidence.extend(pos_low_conf)
    global_neg_high_confidence.extend(neg_high_conf)
    
    print(f"Epoch Final Metrics | "
          f"Train Loss: {train_loss:.4f} | Train AUC: {train_auc:.4f} | Train Acc: {train_acc:.4f} || "
          f"Val Loss: {val_loss:.4f} | Val AUC: {val_auc:.4f} | Val Acc: {val_acc:.4f}")
    print("-" * 40)

# ------------------------------
# Save Low-Confidence Predictions
# ------------------------------
df_pos = meta_data.iloc[global_pos_low_confidence].copy()
df_pos['Issue'] = 'Positive predicted prob < 0.5'
df_neg = meta_data.iloc[global_neg_high_confidence].copy()
df_neg['Issue'] = 'Negative predicted prob >= 0.5'
df_subset = pd.concat([df_pos, df_neg])
df_subset.to_excel(f"low_confidence_{label}_predictions.xlsx", index=False)

# ------------------------------
# Create and Save Summary Metrics DataFrame
# ------------------------------
summary_results = pd.DataFrame({
    "Fold": np.arange(1, n_splits+1),
    "Train Loss": fold_train_losses,
    "Train AUC": fold_train_aucs,
    "Train Accuracy": fold_train_accuracies,
    "Train F1": fold_train_f1s,
    "Train Precision": fold_train_precision,
    "Train Recall": fold_train_recall,
    "Val Loss": fold_val_losses,
    "Val AUC": fold_val_aucs,
    "Val Accuracy": fold_val_accuracies,
    "Val F1": fold_val_f1s,
    "Val Precision": fold_val_precision,
    "Val Recall": fold_val_recall
})
summary_avg_std = pd.DataFrame([{
    "Fold": "Mean ± Std",
    "Train Loss": f"{np.mean(fold_train_losses):.4f} ± {np.std(fold_train_losses):.4f}",
    "Train AUC": f"{np.mean(fold_train_aucs):.4f} ± {np.std(fold_train_aucs):.4f}",
    "Train Accuracy": f"{np.mean(fold_train_accuracies):.4f} ± {np.std(fold_train_accuracies):.4f}",
    "Train F1": f"{np.mean(fold_train_f1s):.4f} ± {np.std(fold_train_f1s):.4f}",
    "Train Precision": f"{np.mean(fold_train_precision):.4f} ± {np.std(fold_train_precision):.4f}",
    "Train Recall": f"{np.mean(fold_train_recall):.4f} ± {np.std(fold_train_recall):.4f}",
    "Val Loss": f"{np.mean(fold_val_losses):.4f} ± {np.std(fold_val_losses):.4f}",
    "Val AUC": f"{np.mean(fold_val_aucs):.4f} ± {np.std(fold_val_aucs):.4f}",
    "Val Accuracy": f"{np.mean(fold_val_accuracies):.4f} ± {np.std(fold_val_accuracies):.4f}",
    "Val F1": f"{np.mean(fold_val_f1s):.4f} ± {np.std(fold_val_f1s):.4f}",
    "Val Precision": f"{np.mean(fold_val_precision):.4f} ± {np.std(fold_val_precision):.4f}",
    "Val Recall": f"{np.mean(fold_val_recall):.4f} ± {np.std(fold_val_recall):.4f}"
}])
summary_results = pd.concat([summary_results, summary_avg_std], ignore_index=True)
summary_results.to_excel(f"{label}_metrics_summary.xlsx", index=False)
print("\n=== Cross-Validation Summary ===")
pd.DataFrame(summary_results)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define common false positive rate grid for interpolation
fpr_common = np.linspace(0, 1, 100)
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

# ------------------------------
# Top Left: Training ROC Curve
# ------------------------------
tprs_train = []
for i, (fpr_i, tpr_i) in enumerate(roc_curves_train):
    interp_tpr = np.interp(fpr_common, fpr_i, tpr_i)
    interp_tpr[0] = 0.0
    tprs_train.append(interp_tpr)
    axs[0, 0].plot(fpr_i, tpr_i, lw=1, alpha=0.3, label=f'Fold {i+1}')
mean_tpr_train = np.mean(tprs_train, axis=0)
std_tpr_train  = np.std(tprs_train, axis=0)
mean_auc_train = np.mean(fold_train_aucs)
std_auc_train  = np.std(fold_train_aucs)
axs[0, 0].plot(fpr_common, mean_tpr_train, color='blue', lw=2,
               label=f'Mean ROC (AUC = {mean_auc_train:.2f} ± {std_auc_train:.2f})')
axs[0, 0].fill_between(fpr_common, mean_tpr_train - std_tpr_train,
                       mean_tpr_train + std_tpr_train, color='blue', alpha=0.2)
axs[0, 0].plot([0, 1], [0, 1], linestyle='--', color='red', label='_nolegend_')
axs[0, 0].set_title('Training ROC Curve')
axs[0, 0].set_xlabel('False Positive Rate')
axs[0, 0].set_ylabel('True Positive Rate')
axs[0, 0].legend(loc="lower right")

# ------------------------------
# Top Right: Validation ROC Curve
# ------------------------------
tprs_val = []
for i, (fpr_i, tpr_i) in enumerate(roc_curves_val):
    interp_tpr = np.interp(fpr_common, fpr_i, tpr_i)
    interp_tpr[0] = 0.0
    tprs_val.append(interp_tpr)
    axs[0, 1].plot(fpr_i, tpr_i, lw=1, alpha=0.3, label=f'Fold {i+1}')
mean_tpr_val = np.mean(tprs_val, axis=0)
std_tpr_val  = np.std(tprs_val, axis=0)
mean_auc_val = np.mean(fold_val_aucs)
std_auc_val  = np.std(fold_val_aucs)
axs[0, 1].plot(fpr_common, mean_tpr_val, color='blue', lw=2,
               label=f'Mean ROC (AUC = {mean_auc_val:.2f} ± {std_auc_val:.2f})')
axs[0, 1].fill_between(fpr_common, mean_tpr_val - std_tpr_val,
                       mean_tpr_val + std_tpr_val, color='blue', alpha=0.2)
axs[0, 1].plot([0, 1], [0, 1], linestyle='--', color='red', label='_nolegend_')
axs[0, 1].set_title('Validation ROC Curve')
axs[0, 1].set_xlabel('False Positive Rate')
axs[0, 1].set_ylabel('True Positive Rate')
axs[0, 1].legend(loc="lower right")

# ------------------------------
# Bottom Left: Training Probability Distribution
# ------------------------------
# Concatenate probabilities and true labels (assumed pre-collected)
train_probs_all = np.concatenate(all_train_probs_1_list)
train_targets_all = np.concatenate(all_train_targets_list)
# Separate probabilities by true label.
train_probs_0 = train_probs_all[train_targets_all == 0]
train_probs_1 = train_probs_all[train_targets_all == 1]

axs[1, 0].hist(train_probs_0, bins=30, alpha=0.6, color='blue', label='True Class 0')
axs[1, 0].hist(train_probs_1, bins=30, alpha=0.6, color='red', label='True Class 1')
axs[1, 0].set_title("Training Set Probability Distribution")
axs[1, 0].set_xlabel("Predicted Probability for Class 1")
axs[1, 0].set_ylabel("Frequency")
axs[1, 0].legend()

# ------------------------------
# Bottom Right: Validation Probability Distribution
# ------------------------------
val_probs_all = np.concatenate(all_val_probs_1_list)
val_targets_all = np.concatenate(all_val_targets_list)
# Separate probabilities by true label.
val_probs_0 = val_probs_all[val_targets_all == 0]
val_probs_1 = val_probs_all[val_targets_all == 1]

axs[1, 1].hist(val_probs_0, bins=30, alpha=0.6, color='blue', label='True Class 0')
axs[1, 1].hist(val_probs_1, bins=30, alpha=0.6, color='red', label='True Class 1')
axs[1, 1].set_title("Validation Set Probability Distribution")
axs[1, 1].set_xlabel("Predicted Probability for Class 1")
axs[1, 1].set_ylabel("Frequency")
axs[1, 1].legend()

plt.tight_layout()
plt.savefig(f"{label}_roc_and_prob.png")
plt.show()

In [ ]:
import json

# Convert NumPy types in the class distribution to native Python types
native_class_distribution = {
    str(k): int(v) if isinstance(v, (np.integer, np.int64)) else v 
    for k, v in class_distribution.items()
}

# Collect hyperparameters in a dictionary
hyperparameters = {
    "input_dim": input_dim,
    "hidden_dim": hidden_dim,
    "output_dim": output_dim,
    "num_epochs": num_epochs,
    "batch_size": batch_size,
    "learning_rate": learning_rate,
    "weight_decay": weight_decay,            # as used in the optimizer
    "lr_scheduler_gamma": gamma,      # gamma for ExponentialLR
    "device": str(device)
}

# Create a dictionary with model and task information
info = {
    "Task": label,                                   # Task/label
    "Classifier": net.module.__name__,          # Classifier name
    "Hyperparameters": hyperparameters,              # Hyperparameters dictionary
    "Class Distribution": native_class_distribution, # Class distribution
    "Cross-Validation Folds": n_splits,              # Number of folds used in CV
    "Metrics Summary": summary_results.to_dict(orient="list")
}

# Write all the information to a text file
with open('Readme.txt', 'w') as f:
    f.write("Model and Task Information\n")
    f.write("==========================\n\n")
    
    f.write("Task: " + str(info["Task"]) + "\n\n")
    f.write("Classifier: " + info["Classifier"] + "\n\n")
    
    f.write("Hyperparameters:\n")
    f.write(json.dumps(info["Hyperparameters"], indent=4))
    f.write("\n\n")
    
    f.write("Class Distribution:\n")
    f.write(json.dumps(info["Class Distribution"], indent=4))
    f.write("\n\n")
    
    f.write("Cross-Validation Folds: " + str(info["Cross-Validation Folds"]) + "\n\n")
    
    f.write("Metrics Summary:\n")
    # Convert summary_results DataFrame to a string for saving
    f.write(summary_results.to_string())
